In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'beomi/OPEN-SOLAR-KO-10.7B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

tokenizer_config.json:   0%|          | 0.00/87.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
generation_config = dict(
    temperature=0.8,
    # top_k=40,
    top_p=0.95,
    # do_sample=True,
    max_new_tokens=512,
    # early_stopping=True,
    # no_repeat_ngram_size=3,
    # eos_token_id=2, 
    # num_beams=1,
    # repetition_penalty=1.1,
)

In [7]:
def gen(x, config, model, tokenizer, device):
    prompt = (
        f"아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{x}\n\n### 응답:"
    )
    len_prompt = len(prompt)
    gened = model.generate(
        **tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(
            device
        ),
        **config
    )
    return tokenizer.decode(gened[0]) # [len_prompt:]

In [9]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

In [11]:
text = "오늘 점심 뭐 먹을까 ?"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?
오늘 점심 뭐 먹을까 ?



In [12]:
output = gen(text, generation_config, model=model, tokenizer=tokenizer, device=device)
output

'<s> 아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n오늘 점심 뭐 먹을까 ?\n\n### 응답:\n오늘은 햄버거가 땡기네\n\n\n오늘 점심 뭐 먹을까 ?\n\n\n오늘은 햄버거가 땡기네</s>'